In [10]:
import requests
import selenium
from selenium import webdriver
import pandas as pd
from random import randint
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select

In [11]:
driver = webdriver.Chrome()
driver.get("https://www.facilito.gob.pe/facilito/pages/facilito/buscadorEESS.jsp") # open Chrome and access this website

In [12]:
driver.find_element(By.XPATH, '//*[@id="Map"]/area[14]').click() # click on the Lima region

In [15]:
def get_dropdown_elements(name): # identify dropdown content
    elements = []
    node = driver.find_element(By.NAME, name)
    places = Select(node)
    for place in places.options[1:]:
        elements.append(place.text)
    return elements 

In [16]:
print(get_dropdown_elements("provincia"), get_dropdown_elements("distrito"), get_dropdown_elements("producto")) # dropdown list elements

provinces = get_dropdown_elements("provincia")

['BARRANCA', 'CAJATAMBO', 'CANTA', 'CAÑETE', 'HUARAL', 'HUAROCHIRI', 'HUAURA', 'LIMA', 'OYON', 'YAUYOS'] [] ['Gasohol Premium', 'Gasohol Regular', 'Gasohol 84 Plus', 'DB5 S-50 UV']


In [17]:
gasoline_types = ["Gasohol Premium", "Gasohol Regular", "Gasohol 84 Plus", "DB5 S-50 UV"]

In [19]:
data = []

for province in provinces:
    Select(driver.find_element(By.NAME, "provincia")).select_by_visible_text(province)
    
    sleep(2)  # delay to allow the page to update

    for gasoline_type in gasoline_types:
        Select(driver.find_element(By.XPATH, '//*[@id="contact"]/div/div[1]/div/div[4]/select')).select_by_visible_text(
            gasoline_type)  # select product
        
        Select(driver.find_element(By.XPATH, '//*[@id="tblPreciosAutomotor_length"]/label/select')).select_by_visible_text(
            '50') # 50 elements per page

        df = pd.read_html(driver.page_source)[0] # read table
        data.append(df)

        sleep(2) #delay to allow the page to update

# concatenate all dataframes into a single dataframe
final_df = pd.concat(data, ignore_index=True)

# CSV file
final_df.to_csv("gasoline_info.csv", index=False)

In [20]:
file = pd.read_csv("gasoline_info.csv")

file

,Distrito,Establecimiento,Dirección,Teléfono,Precio de Venta (Soles por galón)
0,BARRANCA,ESTACION DE SERVICIOS PASO DE LOS ANDES S.A.C.,CARRETERA PANAMERICANA NORTE KM. 193. BARRIO D...,012355428,15.47
1,PARAMONGA,CORPORATION TOCJUN S.A.C.,FRANCISCO VIDAL NRO. 1022,012361343/012352063,15.49
2,BARRANCA,GARCIA ALZAMORA NESTOR JAVIER,LOTE DE TERRENO 259 PAMPAS DE VELARDE,NaN,15.49
3,SUPE PUERTO,PETRO CENTRO SAN PEDRO S.A.C.,BARRIO LA PALMA - ALTURA KM. 191 PANAMERICANA ...,962576413/995615658,15.59
4,BARRANCA,GRIFO BARRANCA VIP S.A.C.,"PARTE DEL SUB LOTE 1 Y SUB LOTE 2, CARRETERA P...",012355014/939342714,15.59
...,...,...,...,...,...
789,OYON,EMPRESA COMUNAL DE SERVICIOS MULTIPLES OYON - ...,BARRIO MUYUCANCHA APROX. DE 2.1 KM. DESDE OYON...,NaN,17.99
790,CATAHUASI,VICENTE SANABRIA JOSE STALIN,CASERIO TAMBO ALT. KM 79 CARRETERA CAÑETE - YA...,NaN,18.6
791,CATAHUASI,VICENTE SANABRIA JOSE STALIN,CASERIO TAMBO ALT. KM 79 CARRETERA CAÑETE - YA...,NaN,17.6
792,No existen registros.,No existen registros.,No existen registros.,No existen registros.,No existen registros.
